In [1]:
parent_folder = '/Users/username/Desktop/ML/Recommendations/cambrian/'
data_folder = parent_folder + 'data/'
from hydra import compose, initialize
with initialize(config_path="../../conf"):
    # Compose the configuration
    cfg = compose(config_name="ClustersResetService.yaml")

import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

/var/folders/d4/cgyr_gnj7nn2wy_hq40gkq8c0000gq/T/ipykernel_68720/1083430745.py:4: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path="../../conf"):


In [186]:
k = {'a': 1}

In [191]:
if [None]:
    print('a')

a


In [2]:
import sys

sys.path.append('/Users/username/Desktop/ML/Recommendations/cambrian/')

import hydra
import os
from hydra import compose, initialize

In [18]:
from pymongo import MongoClient

In [4]:
from sql.clustering.ClusteringSQL import ClusteringSQL
from sql.feed.ArticleSelectionSQL import ArticleSelectionSQL

In [5]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [7]:
import pandas as pd

In [13]:
with PostgresDatabaseOperation() as cursor:
    sql = """SELECT article_id, acm.storyline_id, storyline_name, acm.cluster_id, cluster_name
    FROM article_to_cluster_mapping acm
    LEFT JOIN (SELECT parent_id as storyline_id, parent_name as storyline_name FROM cluster_hierarchy ch) as sl
    ON acm.storyline_id = sl.storyline_id
    LEFT JOIN (SELECT parent_id as cluster_id, parent_name as cluster_name FROM cluster_hierarchy ch) ch 
    ON acm.cluster_id = ch.cluster_id
    """
    cursor.execute(sql)
    result = cursor.fetchall()
    df = pd.DataFrame(result, columns = ['article_id', 'storyline_id','storyline_name', 'cluster_id', 'cluster_name'])

In [14]:
df

,article_id,storyline_id,storyline_name,cluster_id,cluster_name
0,651e19c0a662d76276b8892c,-1,None,191,india_rs_pc_sales_crore
1,651dfda4a662d76276b84616,-1,None,203,date_release_release date_know_season
2,651e222fa662d76276b89d14,96,debt_shutdown_biden_government shutdown_joe biden,194,rs_sebi_rupee_dollar_paise
3,651e1800a662d76276b8852e,60,china_border_ladakh_arunachal_india china,205,ukraine_russia_china_russian_korea
4,651df96aa662d76276b83bae,9,karnataka_congress_bjp_polls_siddaramaiah,201,bjp_manipur_congress_karnataka_pawar
...,...,...,...,...,...
22824,651e11e6a662d76276b876da,31,tour_swift_taylor_taylor swift_concert,203,date_release_release date_know_season
22825,651de7eda662d76276b8102d,-1,None,194,rs_sebi_rupee_dollar_paise
22826,6520c9ddcd871c6ac1cddffd,1,ipl_asian_ipl 2023_games_asian games,185,vs_live_cup_live streaming_streaming
22827,6520c9e7cd871c6ac1cde00a,35,passes away_away_passes_cancer_actor,203,date_release_release date_know_season


In [20]:
class MongoDatabaseConnection:
    _instance = None
    _client = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(MongoDatabaseConnection, cls).__new__(cls)
            cls._instance._client = MongoClient(
                # host='localhost',
                # port=27017,
                host='localhost',
                port=27017
            )
        return cls._instance

    @classmethod
    def get_database(cls):
        db_name = 'insight_db'
        return cls._instance._client[db_name]

In [112]:
def get_article_titles():
    db = MongoDatabaseConnection().get_database()
    articles_collection = db['articles']
    results = articles_collection.find({"is_premium_article": False}, {"published_time": 1, "source_id": 1, "_id": 1, "title": 1, 'url': 1})
    result_list = list(results)
    df = pd.DataFrame(result_list).rename(columns={'_id': 'article_id', 'published_time': 'published_at'})
    df['article_id'] = df['article_id'].apply(lambda x: str(x))
    df['source_id'] = df['source_id'].apply(lambda x: str(x))
    return df

In [113]:
articles= get_article_titles()

In [117]:
is_et = ['https://economictimes.indiatimes.com/' in x for x in articles['url'].values]

In [122]:
articles['category'] = articles['url'].apply(lambda x: x.split('https://economictimes.indiatimes.com/')[1].split('/')[0])
articles['sub_category'] = articles['url'].apply(lambda x: x.split('https://economictimes.indiatimes.com/')[1].split('/')[1])

In [132]:
whitelisted_categories = ['industry', 'markets', 'nri', 'small-biz', 'wealth']
whitelisted_sub_cats = ['news-company', 'news-economy', 'news-industry']

In [133]:
articles['is_relevant'] = articles.apply(lambda x: x['category'] in whitelisted_categories or x['sub_category'] in whitelisted_sub_cats, axis = 1)

In [127]:
articles[articles.sub_category == 'stocks']

,article_id,url,published_at,source_id,title,category,sub_category
93,651dd3d2a662d76276b7dbd5,https://economictimes.indiatimes.com/markets/s...,2023-10-03T19:01:00+05:30,6512cdcad01a9c8e86263e05,Sebi to introduce centralised mechanism for ve...,markets,stocks
159,651dc5fda662d76276b7b573,https://economictimes.indiatimes.com/markets/s...,2023-09-07T20:00:00+05:30,6512cdcad01a9c8e86263e05,Tech shares pressured amid worries over intere...,markets,stocks
194,651dc684a662d76276b7b6bf,https://economictimes.indiatimes.com/markets/s...,2023-09-08T18:53:00+05:30,6512cdcad01a9c8e86263e05,SAT defers hearing on SEBI's confirmatory orde...,markets,stocks
260,651dc736a662d76276b7b8ac,https://economictimes.indiatimes.com/markets/s...,2023-09-09T12:55:00+05:30,6512cdcad01a9c8e86263e05,Why mid-market enterprises explore the AIF rou...,markets,stocks
305,651dc81aa662d76276b7bb43,https://economictimes.indiatimes.com/markets/s...,2023-09-11T10:27:00+05:30,6512cdcad01a9c8e86263e05,"Buy Dalmia Bharat Sugar and Industries, target...",markets,stocks
...,...,...,...,...,...,...,...
22586,651e24dca662d76276b8a38f,https://economictimes.indiatimes.com/markets/s...,2023-04-30T12:28:00+05:30,6512cdcad01a9c8e86263e05,"FPIs invest Rs 11,630 cr in equities in Apr on...",markets,stocks
22650,651e2692a662d76276b8a783,https://economictimes.indiatimes.com/markets/s...,2023-05-03T20:38:00+05:30,6512cdcad01a9c8e86263e05,ABB India net profit falls to Rs 245 crore in ...,markets,stocks
22693,651e259aa662d76276b8a545,https://economictimes.indiatimes.com/markets/s...,2023-05-02T18:51:00+05:30,6512cdcad01a9c8e86263e05,Spandana Sphoorty Q4 Results: PAT rises to Rs ...,markets,stocks
22748,651e269aa662d76276b8a78a,https://economictimes.indiatimes.com/markets/s...,2023-05-02T14:03:00+05:30,6512cdcad01a9c8e86263e05,BP back in Q1 profit after record loss on Russ...,markets,stocks


In [135]:
clusters_df = pd.merge(df, articles, how = 'left', on = 'article_id')

In [138]:
0.99 ** 180

0.16380796970808742

In [139]:
len(clusters_df[clusters_df.is_relevant == True])

3629

In [137]:
clusters_df[(clusters_df.is_relevant == True)].groupby('cluster_name')['article_id'].count()

cluster_name
air_wrestlers_flight_airport_police        310
bjp_manipur_congress_karnataka_pawar        77
case_ed_delhi_sc_court                      24
chandrayaan_isro_mission_space_moon          8
covid_imd_heavy_cases_rains                132
date_release_release date_know_season       79
g20_summit_g20 summit_india_jaishankar      15
india_rs_pc_sales_crore                   1994
modi_pm modi_pm_train_odisha               133
rs_sebi_rupee_dollar_paise                 831
ukraine_russia_china_russian_korea          10
vs_live_cup_live streaming_streaming        16
Name: article_id, dtype: int64

In [102]:
sc_mapping = {row['storyline_id']:row['cluster_id'] for index, row in clusters_df[clusters_df.storyline_id != -1][['storyline_id','cluster_id']].iterrows()}

In [31]:
clusters_df.groupby(['cluster_name','cluster_id'])['article_id'].count()

cluster_name                            cluster_id
air_wrestlers_flight_airport_police     188           1759
bjp_manipur_congress_karnataka_pawar    201           4589
case_ed_delhi_sc_court                  196            949
chandrayaan_isro_mission_space_moon     176            313
covid_imd_heavy_cases_rains             180           1597
date_release_release date_know_season   203           3849
g20_summit_g20 summit_india_jaishankar  146            373
india_rs_pc_sales_crore                 191           3737
modi_pm modi_pm_train_odisha            193            770
pakistan_imran_imran khan_khan_pak      161            278
puzzle_wordle_clues_word_quordle        108            356
rs_sebi_rupee_dollar_paise              194           1424
ukraine_russia_china_russian_korea      205            964
vs_live_cup_live streaming_streaming    185           1871
Name: article_id, dtype: int64

In [37]:
cluster_count_df = clusters_df.groupby(['cluster_id','cluster_name','storyline_id','storyline_name'])['article_id'].count().reset_index()

In [ ]:
cluster_count_df[cluster_count_df.]

In [43]:
cluster_count_df['article_id'].describe([0.1,0.25,0.5,0.75,0.9])

count    108.000000
mean     155.240741
std      145.420748
min       40.000000
10%       48.400000
25%       70.500000
50%       99.500000
75%      180.250000
90%      338.300000
max      869.000000
Name: article_id, dtype: float64

In [74]:
cluster_count_df.storyline_id.nunique()

108

In [90]:
emb = {}

In [91]:
emb['taxation'] = bertopic_model.embedding_model.embed_words('taxation')

In [146]:
storyline_names = {row['storyline_id']:row['storyline_name'] for index, row in cluster_count_df[cluster_count_df.storyline_id != -1].iterrows()}

In [147]:
cluster_names = {row['cluster_id']:row['cluster_name'] for index, row in cluster_count_df[['cluster_id','cluster_name']].drop_duplicates().iterrows()}

In [177]:
def find_storylines_clusters(search_term):
    storyline_ids, prob = bertopic_model.find_topics(search_term,top_n=10)
    valid_indices = [i for i, v in enumerate(storyline_ids) if v != -1]
    storyline_ids = [storyline_ids[i] for i in valid_indices]
    # prob = [prob[i] for i in valid_indices]
    
    s_names = [storyline_names[sc_id] for sc_id in storyline_ids]
    cluster_ids = [sc_mapping[sc_id] for sc_id in storyline_ids]
    c_names = [cluster_names[c_id] for c_id in cluster_ids]
    # print(storyline_ids, s_names, cluster_ids, c_names)
    mapping = {storyline_ids[i]: {'storyline_name': s_names[i], 
                        'cluster_name': c_names[i],
                       'prob': prob[i],
                       'cluster_id': cluster_ids[i]} for i in range(len(storyline_ids))}
    return mapping

In [184]:
find_storylines_clusters('investment: real estate investment')

{53: {'storyline_name': 'housing_cr_rs_realty_real estate',
  'cluster_name': 'india_rs_pc_sales_crore',
  'prob': 0.6912444,
  'cluster_id': 191},
 80: {'storyline_name': 'stocks_52 week_52_nifty_week',
  'cluster_name': 'rs_sebi_rupee_dollar_paise',
  'prob': 0.5862149,
  'cluster_id': 194},
 89: {'storyline_name': 'china_economic_economy_china economy_economy china',
  'cluster_name': 'ukraine_russia_china_russian_korea',
  'prob': 0.5817936,
  'cluster_id': 205},
 85: {'storyline_name': 'growth_fy24_revenue_fmcg_growth fy24',
  'cluster_name': 'india_rs_pc_sales_crore',
  'prob': 0.5806071,
  'cluster_id': 191},
 73: {'storyline_name': 'ipo_papers_sebi_papers sebi_rs',
  'cluster_name': 'rs_sebi_rupee_dollar_paise',
  'prob': 0.58016825,
  'cluster_id': 194},
 93: {'storyline_name': 'crypto_bitcoin_ethereum_everlodge_pomerdoge',
  'cluster_name': 'rs_sebi_rupee_dollar_paise',
  'prob': 0.5659201,
  'cluster_id': 194},
 18: {'storyline_name': 'profit_bank_rbi_results_net',
  'cluste

In [183]:
find_storylines_clusters('investment: real estate and home ownership')

{53: {'storyline_name': 'housing_cr_rs_realty_real estate',
  'cluster_name': 'india_rs_pc_sales_crore',
  'prob': 0.6690377,
  'cluster_id': 191},
 43: {'storyline_name': 'fed_inflation_rate_rates_economy',
  'cluster_name': 'india_rs_pc_sales_crore',
  'prob': 0.6184423,
  'cluster_id': 191},
 89: {'storyline_name': 'china_economic_economy_china economy_economy china',
  'cluster_name': 'ukraine_russia_china_russian_korea',
  'prob': 0.6152699,
  'cluster_id': 205},
 18: {'storyline_name': 'profit_bank_rbi_results_net',
  'cluster_name': 'india_rs_pc_sales_crore',
  'prob': 0.5913097,
  'cluster_id': 191},
 80: {'storyline_name': 'stocks_52 week_52_nifty_week',
  'cluster_name': 'rs_sebi_rupee_dollar_paise',
  'prob': 0.5877895,
  'cluster_id': 194},
 99: {'storyline_name': 'irdai_insurance_insurers_life_lic',
  'cluster_name': 'rs_sebi_rupee_dollar_paise',
  'prob': 0.57964313,
  'cluster_id': 194},
 84: {'storyline_name': 'growth_rbi_gdp_cea_economy',
  'cluster_name': 'india_rs_pc

In [178]:
find_storylines_clusters('taxation')

{47: {'storyline_name': 'gst_tax_evasion_income tax_gaming',
  'cluster_name': 'rs_sebi_rupee_dollar_paise',
  'prob': 0.6482755,
  'cluster_id': 194},
 29: {'storyline_name': 'rice_tomatoes_prices_wheat_kg',
  'cluster_name': 'india_rs_pc_sales_crore',
  'prob': 0.5816685,
  'cluster_id': 191},
 26: {'storyline_name': 'sc_cji_marriage_sex_court',
  'cluster_name': 'case_ed_delhi_sc_court',
  'prob': 0.56120294,
  'cluster_id': 196},
 14: {'storyline_name': 'modi_pm_pm modi_projects_rs',
  'cluster_name': 'modi_pm modi_pm_train_odisha',
  'prob': 0.5608822,
  'cluster_id': 193},
 18: {'storyline_name': 'profit_bank_rbi_results_net',
  'cluster_name': 'india_rs_pc_sales_crore',
  'prob': 0.5557394,
  'cluster_id': 191},
 43: {'storyline_name': 'fed_inflation_rate_rates_economy',
  'cluster_name': 'india_rs_pc_sales_crore',
  'prob': 0.5551454,
  'cluster_id': 191},
 17: {'storyline_name': 'exports_trade_commerce_india_goyal',
  'cluster_name': 'india_rs_pc_sales_crore',
  'prob': 0.5549

In [87]:
clusters_df[clusters_df.storyline_id == 47]

,article_id,storyline_id,storyline_name,cluster_id,cluster_name,published_at,source_id,title
180,651deef8a662d76276b822a0,47,gst_tax_evasion_income tax_gaming,194,rs_sebi_rupee_dollar_paise,2023-07-25T18:31:00+05:30,6512cdcad01a9c8e86263e05,"Over 8.75 lakh HUFs claimed Rs 3,803 crore tax..."
347,651de11ba662d76276b7fe18,47,gst_tax_evasion_income tax_gaming,194,rs_sebi_rupee_dollar_paise,2023-08-30T20:13:00+05:30,6512cdcad01a9c8e86263e05,GST fraud: Taxmen arrest two in Bengaluru for ...
579,651dcfd1a662d76276b7d062,47,gst_tax_evasion_income tax_gaming,194,rs_sebi_rupee_dollar_paise,2023-09-27T10:38:00+05:30,6512cdcad01a9c8e86263e05,No relief in Rs 10 lakh tax fine for not discl...
585,651e17f3a662d76276b8850a,47,gst_tax_evasion_income tax_gaming,194,rs_sebi_rupee_dollar_paise,2023-04-11T15:17:00+05:30,6512cdcad01a9c8e86263e05,"If fantasy games are OK, why not online betting?"
862,651e209ea662d76276b89963,47,gst_tax_evasion_income tax_gaming,194,rs_sebi_rupee_dollar_paise,2023-04-23T09:56:00+05:30,6512cdcad01a9c8e86263e05,GST fraud a new challenge for UP government; N...
...,...,...,...,...,...,...,...,...
22089,651e180ca662d76276b88548,47,gst_tax_evasion_income tax_gaming,194,rs_sebi_rupee_dollar_paise,2023-04-11T16:35:00+05:30,6512cdcad01a9c8e86263e05,Karnataka: I-T dept detects Rs 1k cr bogus exp...
22194,651e17c8a662d76276b884a5,47,gst_tax_evasion_income tax_gaming,194,rs_sebi_rupee_dollar_paise,2023-04-06T21:09:00+05:30,6512cdcad01a9c8e86263e05,"Government releases norms for online gaming, p..."
22269,651df244a662d76276b82b43,47,gst_tax_evasion_income tax_gaming,194,rs_sebi_rupee_dollar_paise,2023-08-01T16:08:00+05:30,6512cdcad01a9c8e86263e05,Sufficient enforcement provisions in law to ta...
22334,651e259ea662d76276b8a551,47,gst_tax_evasion_income tax_gaming,194,rs_sebi_rupee_dollar_paise,2023-05-02T17:52:00+05:30,6512cdcad01a9c8e86263e05,Centre asks states to crack down on outdoor ad...


In [109]:
cluster_count_df[cluster_count_df.cluster_id == 203]

,cluster_id,cluster_name,storyline_id,storyline_name,article_id
87,203,date_release_release date_know_season,0,season_netflix_release_release date_date,869
88,203,date_release_release date_know_season,7,film_khan_ott_collection_office collection,415
89,203,date_release_release date_know_season,16,result_class_results_2023_result 2023,260
90,203,date_release_release date_know_season,21,xbox_game_diablo_release_nintendo,195
91,203,date_release_release date_know_season,31,tour_swift_taylor_taylor swift_concert,171
92,203,date_release_release date_know_season,32,day_day 2023_significance_2023_history,165
93,203,date_release_release date_know_season,35,passes away_away_passes_cancer_actor,155
94,203,date_release_release date_know_season,40,step_apple_musk_guide_step guide,130
95,203,date_release_release date_know_season,44,divorce_spears_britney spears_britney_know,120
96,203,date_release_release date_know_season,57,barbie_oppenheimer_box_box office_office,98


In [110]:
cluster_count_df[cluster_count_df.cluster_id == 203].article_id.sum()

2877

In [47]:
from src._utils import load_bertopic_model_from_hf
bertopic_model = load_bertopic_model_from_hf()

/Users/username/anaconda3/envs/recoenv/lib/python3.11/site-packages/bertopic/_bertopic.py:3980: UserWarning: You are loading a BERTopic model without explicitly defining an embedding model.If you want to also load in an embedding model, make sure to useBERTopic.load(my_model, embedding_model=my_embedding_model).
  warnings.warn("You are loading a BERTopic model without explicitly defining an embedding model."


In [82]:
from sklearn.metrics.pairwise import cosine_similarity, manhattan_distances

In [83]:
def get_cs_between_storylines(id1, id2, dist = 'cos'):
    if dist == 'cos':
        return cosine_similarity(bertopic_model.topic_embeddings_[id1].reshape(-1,1).T, bertopic_model.topic_embeddings_[id2].reshape(-1,1).T)
    else:
        return manhattan_distances(bertopic_model.topic_embeddings_[id1].reshape(-1,1).T, bertopic_model.topic_embeddings_[id2].reshape(-1,1).T)

In [60]:
cosine_similarity(bertopic_model.topic_embeddings_[30].reshape(-1,1).T, bertopic_model.topic_embeddings_[38].reshape(-1,1).T)

array([[0.80502236]], dtype=float32)

In [85]:
get_cs_between_storylines(47, 45, dist = 'man')

array([[17.21528239]])

In [86]:
cluster_count_df[cluster_count_df.storyline_id == 1]

,cluster_id,cluster_name,storyline_id,storyline_name,article_id
16,185,vs_live_cup_live streaming_streaming,1,ipl_asian_ipl 2023_games_asian games,805


array([ 0.0206767 , -0.01522607, -0.02201222, ..., -0.01409806,
       -0.00595029, -0.01660891], dtype=float32)

array([-0.00923504,  0.0186849 , -0.01047261, ..., -0.00727443,
       -0.02175595, -0.00562101], dtype=float32)

In [42]:
clusters_df[(clusters_df.cluster_id == 191) & (clusters_df.storyline_id == -1)].sample(20)['title'].values

array(["Regulator's review of over 30 Swiss banks finds shortcomings in money-laundering controls",
       'China names Pan Gongsheng to lead central bank, succeeding Yi Gang',
       'Rahul Gandhi has deep understanding of technology, says entrepreneur who hosted him in Silicon Valley',
       "Kolkata Port profit jumps 152pc in FY'23; trial run to Myanmar's Sittwe port in April",
       'When Narayana Murthy mentioned about Alia Bhatt in his address to IIM students',
       "Singapore's total population touch new high at 5.92 million: Government",
       'Adequate availability of fertilisers for kharif sowing; states must act against diversion of urea: Mandavi',
       'PM Modi, British counterpart Rishi Sunak review bilateral strategic ties',
       'UK PM Rishi Sunak defends actions over school closures',
       'Palm oil imports fall 15% to 4.39 lakh tonnes in May',
       'SBI Card plans to raise Rs 3,000 cr from debentures',
       'Australia must continue to expand practical co

In [ ]:
clusters_df[clusters_df.cluster_id == 191]

In [36]:
clusters_df[clusters_df.cluster_id == 191][['storyline_name', 'cluster_name']].drop_duplicates()

,storyline_name,cluster_name
0,None,india_rs_pc_sales_crore
17,rice_tomatoes_prices_wheat_kg,india_rs_pc_sales_crore
26,power_energy_mw_solar_renewable,india_rs_pc_sales_crore
46,electric_sales_ev_units_motors,india_rs_pc_sales_crore
85,steel_coal_mt_tata steel_coal india,india_rs_pc_sales_crore
96,profit_bank_rbi_results_net,india_rs_pc_sales_crore
120,fed_inflation_rate_rates_economy,india_rs_pc_sales_crore
186,housing_cr_rs_realty_real estate,india_rs_pc_sales_crore
188,exports_trade_commerce_india_goyal,india_rs_pc_sales_crore
254,oil_gas_ongc_prices_lpg,india_rs_pc_sales_crore
